# 🧠 Notebook d'Analyse Rhétorique Collaborative par Agents IA (Modulaire - v2)

Bienvenue dans ce notebook utilisant Semantic Kernel pour orchestrer une analyse rhétorique collaborative. Plusieurs agents spécialisés vont travailler ensemble pour analyser un texte fourni. Cette version est structurée de manière modulaire, avec des cellules dédiées pour chaque agent et leurs composants.

**Objectif :** Analyser un texte sous différents angles (informel et formel simple via logique propositionnelle) en observant la collaboration des agents via la modification d'un état partagé. Utiliser une orchestration basée sur la désignation explicite de l'agent suivant via l'état.

**Structure :**
1.  Configuration Initiale & Dépendances (Python, LLM)
2.  Configuration Java/Tweety (pour l'analyse logique formelle)
3.  Définitions des Composants Partagés (État, StateManager, Service LLM Global)
4.  Agent: Project Manager (Définitions)
5.  Agent: Informal Analysis (Définitions)
6.  Agent: Propositional Logic (Définitions)
7.  Orchestration de la Conversation (Définitions des Stratégies)
8.  Exécution de la Conversation Collaborative (Instanciation et Lancement)
9.  Conclusion & Prochaines Étapes

*(Version 2 : Correction bugs désignation/affichage agent, validation type logique, nettoyage code et amélioration documentation)*

## 1. Configuration Initiale et Dépendances (Python, LLM)

Cette cellule unique regroupe :
*   L'installation et la vérification des dépendances Python nécessaires (`semantic-kernel`, `jpype1`, `python-dotenv`, `pandas`, `requests`).
*   La configuration du logging global.
*   Le chargement de la configuration du LLM (OpenAI ou Azure OpenAI) depuis le fichier `.env`. **Assurez-vous d'avoir un fichier `.env` à la racine avec vos clés API et identifiants de modèle.**
*   La définition du texte source (`raw_text_input`) qui sera analysé par les agents.

In [ ]:
# %% CELLULE [1] MODIFIÉE (ID 25d83fde) - Configuration Initiale et Dépendances (Python, LLM)
# Regroupe dépendances, config LLM. *** raw_text_input a été SUPPRIMÉ ***

%pip install --upgrade semantic-kernel python-dotenv ipywidgets jpype1 requests tqdm pandas 
# pandas ajouté pour plugin informel
# Décommentez la ligne ci-dessus si les packages ne sont pas déjà installés

import sys
import importlib
import subprocess
import os
from dotenv import load_dotenv
import logging

# --- Configuration du Logging Global ---
# Format amélioré incluant le nom du logger pour mieux tracer
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] [%(name)s] %(message)s', datefmt='%H:%M:%S')
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("semantic_kernel.connectors.ai").setLevel(logging.WARNING)
logging.getLogger("semantic_kernel.kernel").setLevel(logging.WARNING)
logging.getLogger("semantic_kernel.functions").setLevel(logging.WARNING)
# Garder INFO pour les agents et l'orchestration pour suivre le déroulement
logging.getLogger("semantic_kernel.agents").setLevel(logging.INFO)
logging.getLogger("Orchestration").setLevel(logging.INFO)
# Configurer les loggers spécifiques utilisés plus loin si nécessaire
# (Ex: logging.getLogger("Orchestration.AgentPM").setLevel(logging.DEBUG) pour plus de détails sur un agent)

logger = logging.getLogger("Orchestration.Setup") # Logger pour cette cellule

# --- Vérification et Installation Dépendances ---
def check_and_install(package_import_name: str, package_install_name: str):
    """Vérifie si un package est importable, sinon tente de l'installer."""
    try:
        importlib.import_module(package_import_name)
        logger.info(f"✔️ Dépendance '{package_import_name}' trouvée.")
        return True
    except ImportError:
        logger.warning(f"⚠️ Dépendance '{package_import_name}' manquante (package: {package_install_name}). Tentative d'installation...")
        try:
            # Utilisation de -q pour une sortie moins verbeuse, --disable-pip-version-check pour éviter les warnings
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--disable-pip-version-check", package_install_name])
            logger.info(f"✅ {package_install_name} installé avec succès.")
            # Recharger les modules ou invalider les caches peut être nécessaire dans certains environnements
            importlib.invalidate_caches()
            importlib.import_module(package_import_name) # Re-tester l'import
            logger.info(f"✔️ {package_import_name} trouvé après installation.")
            return True
        except Exception as e:
            logger.error(f"❌ Échec de l'installation/import de {package_install_name}: {e}")
            logger.warning("‼️ Un redémarrage du noyau (Kernel -> Restart Kernel) peut être nécessaire si l'import échoue toujours.")
            return False

logger.info("--- Vérification des dépendances ---")
deps_ok = True
deps_list = [
    ("jpype", "jpype1"),
    ("semantic_kernel", "semantic-kernel"),
    ("dotenv", "python-dotenv"),
    ("pandas", "pandas"), # Nécessaire pour InformalAnalysisPlugin
    ("requests", "requests") # Nécessaire pour InformalAnalysisPlugin (téléchargement CSV)
]
for import_name, install_name in deps_list:
    if not check_and_install(import_name, install_name):
        deps_ok = False

if not deps_ok:
    logger.critical("\n❌ Des dépendances clés sont manquantes ou n'ont pu être importées après installation. Veuillez vérifier les erreurs et redémarrer le noyau si nécessaire avant de continuer.")
    # Optionnel: Lever une exception pour arrêter l'exécution
    # raise RuntimeError("Dépendances manquantes ou nécessitant un redémarrage du noyau.")
else:
    logger.info("\n✅ Dépendances principales vérifiées.")

# --- Chargement config LLM depuis .env ---
logger.info("--- Chargement Configuration LLM ---")
load_dotenv(override=True) # `override=True` pour recharger si nécessaire

api_key = os.getenv("OPENAI_API_KEY")
model_id = os.getenv("OPENAI_CHAT_MODEL_ID")
endpoint = os.getenv("OPENAI_ENDPOINT") # Endpoint spécifique Azure
org_id = os.getenv("OPENAI_ORG_ID") # Optionnel pour OpenAI standard
use_azure_openai = bool(endpoint) # Détermine si on utilise Azure en fonction de la présence de l'endpoint

llm_configured = False
if use_azure_openai:
    # Valider la configuration Azure
    if not all([api_key, model_id, endpoint]):
        logger.error("❌ Configuration Azure OpenAI incomplète dans .env (OPENAI_API_KEY, OPENAI_CHAT_MODEL_ID, OPENAI_ENDPOINT requis).")
    else:
        logger.info(f"✅ Configuration Azure OpenAI détectée (Deployment: {model_id}, Endpoint: {endpoint[:20]}...).")
        llm_configured = True
else:
    # Valider la configuration OpenAI standard
    if not all([api_key, model_id]):
            logger.error("❌ Configuration OpenAI standard incomplète dans .env (OPENAI_API_KEY, OPENAI_CHAT_MODEL_ID requis).")
    else:
        logger.info(f"✅ Configuration OpenAI standard détectée (Modèle: {model_id}). Org ID: {'Fourni' if org_id else 'Non fourni'}.")
        llm_configured = True

if not llm_configured:
    raise ValueError("Configuration LLM échouée. Vérifiez votre fichier .env et les logs ci-dessus.")


# Assurer que la variable use_azure_openai existe pour les cellules suivantes, même si la config a échoué plus tôt
if 'use_azure_openai' not in locals():
    use_azure_openai = False # Défaut
    logger.warning("Variable 'use_azure_openai' non définie (erreur config?), défaut à False.")

logger.info("--- Fin Configuration Initiale ---")

### Configuration du LLM (via .env)

Assurez-vous d'avoir un fichier `.env` avec vos identifiants LLM (voir exemple dans le notebook générateur ou la cellule précédente).

In [ ]:
# %% Chargement de la configuration LLM depuis .env
import os
from dotenv import load_dotenv
import logging # Ajouter logging pour info

# Configurer un logger simple si besoin
cfg_logger = logging.getLogger("Orchestration.Config")
if not cfg_logger.handlers and not cfg_logger.propagate:
     handler = logging.StreamHandler(); formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(name)s - %(message)s', datefmt='%H:%M:%S'); handler.setFormatter(formatter); cfg_logger.addHandler(handler); cfg_logger.setLevel(logging.INFO)

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
model_id = os.getenv("OPENAI_CHAT_MODEL_ID")
endpoint = os.getenv("OPENAI_ENDPOINT") # Endpoint spécifique Azure
org_id = os.getenv("OPENAI_ORG_ID") # Optionnel pour OpenAI standard

# --- Définition de use_azure_openai ---
use_azure_openai = bool(endpoint)
# ---------------------------------------

# Vérifications et logs
if use_azure_openai:
    if not all([api_key, model_id, endpoint]):
        msg = "⚠️ Configuration Azure OpenAI incomplète dans .env (OPENAI_API_KEY, OPENAI_CHAT_MODEL_ID, OPENAI_ENDPOINT requis)."
        cfg_logger.warning(msg)
        print(msg)
    else:
         msg = f"✅ Configuration Azure OpenAI chargée (Deployment: {model_id}, Endpoint: {endpoint[:20]}...)."
         cfg_logger.info(msg)
         print(msg)
else:
    if not all([api_key, model_id]):
         msg = "⚠️ Configuration OpenAI standard incomplète dans .env (OPENAI_API_KEY, OPENAI_CHAT_MODEL_ID requis)."
         cfg_logger.warning(msg)
         print(msg)
    else:
        msg = f"✅ Configuration OpenAI standard chargée (Modèle: {model_id}). Org ID: {'Fourni' if org_id else 'Non fourni'}."
        cfg_logger.info(msg)
        print(msg)

# S'assurer que la variable existe même si la config est incomplète, pour éviter NameError
if 'use_azure_openai' not in locals():
    use_azure_openai = False # Défaut si erreur précédente
    cfg_logger.warning("Variable 'use_azure_openai' non définie due à une erreur de config, défaut à False.")

## 2. Configuration de l'environnement Java/Tweety (JPype)

Cette section est **cruciale** pour utiliser les fonctionnalités d'analyse logique formelle via Tweety (utilisées par le `PropositionalLogicAgent`).

**Prérequis INDISPENSABLES :**

1.  **Installation d'un JDK :** Vous devez avoir un Java Development Kit (JDK) version 11 ou supérieure installé sur votre système.
2.  **Configuration de `JAVA_HOME` :** La variable d'environnement `JAVA_HOME` **doit pointer vers le répertoire racine de votre installation JDK**. C'est la méthode la plus fiable pour que JPype trouve la JVM.
    *   **Windows :** ex: `C:\Program Files\Java\jdk-17` (Adaptez). Ajoutez aux variables d'environnement système/utilisateur.
    *   **Linux/macOS :** ex: `/usr/lib/jvm/java-17-openjdk-amd64` ou `/Library/Java/JavaVirtualMachines/zulu-17.jdk/Contents/Home`. Ajoutez `export JAVA_HOME=/chemin/vers/jdk` à votre `~/.bashrc`, `~/.zshrc` ou profil.
    *   **Redémarrage OBLIGATOIRE :** Après avoir défini `JAVA_HOME`, **redémarrez votre environnement Jupyter** (serveur JupyterLab/Notebook ET le noyau de ce notebook) pour qu'elle soit prise en compte.
3.  **JARs Tweety :** Les fichiers `.jar` de Tweety (au moins `tweety-full-...jar` et les modules comme `logics.pl-...jar`) doivent être dans le dossier `./libs/` (ou le chemin configuré dans la cellule suivante).

La cellule suivante tentera de démarrer la JVM via `jpype`. Elle utilise une fonction `find_java_home` pour vérifier `JAVA_HOME` et tenter une détection automatique, mais **la définition manuelle de `JAVA_HOME` est fortement recommandée.** La variable globale `jvm_ready` indiquera si le démarrage a réussi.

In [ ]:
# %% CELLULE OPTION B - Configuration Java Auto-Suffisante (Initialisation Embarquée)
# Intégration des fonctions validées des scripts de réparation pour rendre le notebook autonome

import jpype
import jpype.imports
import os
import pathlib
import platform
import sys
import logging
from typing import Optional

logger = logging.getLogger("Orchestration.JPype")
logger.info("\n--- Configuration Auto-Suffisante de la JVM (Option B - Embarquée) ---")

# --- Fonctions embarquées (extraites des scripts de réparation validés) ---

def find_portable_jdk() -> Optional[str]:
    """
    Localise automatiquement le JDK portable dans l'arborescence du projet.
    Fonction extraite et validée du script de réparation.
    """
    logger.info("🔍 Recherche JDK portable dans l'arborescence projet...")
    
    # Chemins de recherche prioritaires pour le JDK portable
    search_paths = [
        pathlib.Path("MyIA.AI.Notebooks/SymbolicAI/Argument_Analysis/jdk-17-portable"),
        pathlib.Path("jdk-17-portable"),
        pathlib.Path("../jdk-17-portable"),
        pathlib.Path("../../jdk-17-portable")
    ]
    
    for base_path in search_paths:
        if base_path.exists():
            logger.debug(f"  Scan du répertoire: {base_path}")
            
            # Chercher des sous-dossiers JDK
            jdk_patterns = ["*jdk*", "zulu*", "*openjdk*", "*corretto*"]
            for pattern in jdk_patterns:
                jdk_dirs = list(base_path.glob(pattern))
                for jdk_dir in jdk_dirs:
                    if jdk_dir.is_dir():
                        # Vérifier présence de bin/java
                        exe_suffix = ".exe" if platform.system() == "Windows" else ""
                        java_exe = jdk_dir / "bin" / f"java{exe_suffix}"
                        if java_exe.exists():
                            logger.info(f"✅ JDK portable trouvé: {jdk_dir.absolute()}")
                            return str(jdk_dir.absolute())
    
    logger.warning("⚠️ JDK portable non trouvé dans les chemins standards")
    return None

def get_tweety_classpath() -> list:
    """
    Construit dynamiquement le classpath avec tous les JARs Tweety disponibles.
    Fonction extraite et validée du script de réparation.
    """
    logger.info("📦 Construction du classpath Tweety...")
    
    # Chemins de recherche des JARs
    libs_paths = [
        pathlib.Path("libs"),
        pathlib.Path("MyIA.AI.Notebooks/SymbolicAI/libs"),
        pathlib.Path("../libs")
    ]
    
    jar_files = []
    
    for libs_path in libs_paths:
        if libs_path.exists() and libs_path.is_dir():
            logger.info(f"  Scan des JARs dans: {libs_path}")
            found_jars = list(libs_path.glob("*.jar"))
            if found_jars:
                jar_files.extend([str(jar.absolute()) for jar in found_jars])
                logger.info(f"  ✅ {len(found_jars)} JARs trouvés dans {libs_path}")
                break  # Utiliser le premier répertoire avec des JARs
    
    if jar_files:
        logger.info(f"✅ Classpath construit: {len(jar_files)} JARs Tweety")
        return sorted(jar_files)  # Tri pour cohérence
    else:
        logger.error("❌ Aucun JAR Tweety trouvé dans les chemins standards")
        return []

def test_tweety_critical_classes():
    """
    Teste l'accès aux classes critiques de Tweety pour validation.
    Utilise la classe corrigée découverte lors du debug.
    """
    logger.info("🎯 Test des classes critiques Tweety...")
    
    critical_classes = {
        "InformationObject": "org.tweetyproject.beliefdynamics.mas.InformationObject",
        "PlParser": "org.tweetyproject.logics.pl.parser.PlParser",
        "PlFormula": "org.tweetyproject.logics.pl.syntax.PlFormula",
        "BaseRevisionOperator": "org.tweetyproject.beliefdynamics.BaseRevisionOperator"
    }
    
    successful_classes = []
    
    for class_alias, class_name in critical_classes.items():
        try:
            test_class = jpype.JClass(class_name)
            successful_classes.append(class_alias)
            logger.info(f"  ✅ {class_alias}: Accessible")
        except Exception as e:
            logger.warning(f"  ⚠️ {class_alias}: Inaccessible - {e}")
    
    success_rate = len(successful_classes) / len(critical_classes)
    
    if success_rate >= 0.75:  # Au moins 75% des classes critiques
        logger.info(f"🏆 Test Tweety RÉUSSI: {len(successful_classes)}/{len(critical_classes)} classes accessibles")
        return True
    else:
        logger.error(f"❌ Test Tweety ÉCHOUÉ: {len(successful_classes)}/{len(critical_classes)} classes accessibles")
        return False

# --- Séquence d'initialisation embarquée ---

jvm_ready = False  # Variable globale d'état

try:
    # Étape 1: Vérifier si JVM déjà démarrée
    if jpype.isJVMStarted():
        logger.info("ℹ️ JVM déjà démarrée - utilisation de l'instance existante")
        jvm_ready = True
        
        # Enregistrer les domaines au cas où
        try:
            jpype.imports.registerDomain("org", alias="org")
            jpype.imports.registerDomain("java", alias="java")
            jpype.imports.registerDomain("net", alias="net")
        except:
            pass
    else:
        # Étape 2: Localiser et configurer JAVA_HOME au niveau processus
        portable_jdk = find_portable_jdk()
        
        if portable_jdk:
            # Configuration dynamique de JAVA_HOME pour ce processus
            current_java_home = os.getenv('JAVA_HOME')
            if not current_java_home or current_java_home != portable_jdk:
                os.environ['JAVA_HOME'] = portable_jdk
                logger.info(f"🏠 JAVA_HOME configuré dynamiquement: {portable_jdk}")
            else:
                logger.info(f"🏠 JAVA_HOME déjà configuré: {current_java_home}")
        else:
            logger.error("❌ Impossible de localiser un JDK portable - JVM pourrait échouer")
        
        # Étape 3: Construire le classpath Tweety complet
        tweety_jars = get_tweety_classpath()
        
        if not tweety_jars:
            raise Exception("Classpath Tweety vide - JARs manquants")
        
        # Étape 4: Démarrer la JVM avec configuration optimale
        logger.info(f"🚀 Démarrage JVM avec {len(tweety_jars)} JARs Tweety...")
        
        jvm_args = [
            "-Xmx2g",  # Mémoire suffisante pour Tweety
            "-Djava.awt.headless=true",  # Mode headless pour notebooks
        ]
        
        jpype.startJVM(
            *jvm_args,
            classpath=tweety_jars,
            convertStrings=False,
            ignoreUnrecognized=True
        )
        
        # Étape 5: Enregistrer les domaines Java
        jpype.imports.registerDomain("org", alias="org")
        jpype.imports.registerDomain("java", alias="java")
        jpype.imports.registerDomain("net", alias="net")
        
        logger.info("✅ JVM démarrée avec succès et domaines enregistrés")
        jvm_ready = True
    
    # Étape 6: Test de validation Tweety
    if jvm_ready:
        # Test Java de base
        try:
            System = jpype.JClass('java.lang.System')
            java_version = System.getProperty('java.version')
            logger.info(f"☕ Java {java_version} opérationnel")
        except Exception as e:
            logger.warning(f"⚠️ Test Java de base échoué: {e}")
        
        # Test classes Tweety critiques
        tweety_success = test_tweety_critical_classes()
        if tweety_success:
            logger.info("\n🎉 🏆 SUCCÈS TOTAL - INFRASTRUCTURE TWEETY AUTO-SUFFISANTE! 🏆 🎉")
            logger.info("✅ PropositionalLogicAgent prêt pour exécution native")
            logger.info("✅ Pipeline argumentatif avec intégration formelle/informelle opérationnel")
        else:
            logger.warning("⚠️ JVM opérationnelle mais classes Tweety partiellement accessibles")
            jvm_ready = False

except Exception as e:
    logger.critical(f"❌ ERREUR CRITIQUE Configuration Java: {e}")
    logger.critical("   PropositionalLogicAgent fonctionnera en mode dégradé (LLM seulement)")
    jvm_ready = False

# --- Conclusion État JVM ---
if jvm_ready:
    logger.info("\n🟢 STATUT FINAL: JVM + Tweety OPÉRATIONNELS")
    logger.info("   Pipeline peut atteindre son potentiel maximal (8/10)")
else:
    logger.warning("\n🔴 STATUT FINAL: JVM/Tweety NON OPÉRATIONNELS")
    logger.warning("   Pipeline fonctionnera en mode dégradé (5/10)")

logger.info("--- Fin Configuration JPype Auto-Suffisante ---")

# === FONCTIONS UTILITAIRES INTÉGRÉES ===
# Ces fonctions sont extraites et optimisées des scripts de diagnostic/réparation

def diagnostic_infrastructure_tweety():
    """Fonction de diagnostic complet de l'infrastructure Tweety intégrée."""
    logger.info("🔍 Diagnostic infrastructure Tweety intégré...")
    
    diagnostic_results = {
        'jdk_status': 'OK' if find_portable_jdk() else 'NOK',
        'jars_count': len(get_tweety_classpath()),
        'jvm_ready': jvm_ready,
        'java_version': None
    }
    
    if jvm_ready:
        try:
            System = jpype.JClass('java.lang.System')
            diagnostic_results['java_version'] = System.getProperty('java.version')
        except:
            pass
    
    logger.info(f"📊 Résultats: {diagnostic_results}")
    return diagnostic_results

def validate_tweety_ready():
    """Validation rapide de l'état opérationnel Tweety."""
    if not jvm_ready:
        logger.warning("⚠️ JVM non prête - PropositionalLogicAgent en mode dégradé")
        return False
    
    try:
        # Test classe critique
        InformationObject = jpype.JClass('org.tweetyproject.beliefdynamics.mas.InformationObject')
        PlParser = jpype.JClass('org.tweetyproject.logics.pl.parser.PlParser')
        logger.info("✅ Tweety opérationnel - PropositionalLogicAgent en mode natif")
        return True
    except Exception as e:
        logger.error(f"❌ Classes Tweety inaccessibles: {e}")
        return False

# Diagnostic automatique à l'initialisation
if 'jvm_ready' in locals():
    validate_tweety_ready()

## 3. Définitions des Composants Partagés

Cette section définit les **classes** et le **service LLM global** utilisés par plusieurs agents. Les **instances** spécifiques (état, StateManager, kernel local, agents) seront créées plus tard, dans la fonction d'exécution (Section 8).

*   **`RhetoricalAnalysisState` (Classe)** : La classe Python représentant l'état partagé de l'analyse (texte brut, tâches, arguments, sophismes, belief sets, réponses, etc.). Inclut maintenant un logging interne plus détaillé.
*   **`StateManagerPlugin` (Classe)** : Le plugin Semantic Kernel qui fournit des fonctions (`@kernel_function`) pour lire et modifier une instance de `RhetoricalAnalysisState`. Sera initialisé avec l'instance d'état locale lors de l'exécution.
*   **`global_ai_service_instance` (Instance)** : L'instance unique du service de complétion (OpenAI ou Azure) configurée globalement. Elle sera ajoutée au kernel *local* de chaque agent lors de sa création.

### 🧱 Classe : RhetoricalAnalysisState

In [ ]:
# %% CELLULE [3.1] - Définition Classe RhetoricalAnalysisState
# (Remplace une partie de l'ancienne cellule 24085a21)

import json
from typing import Dict, List, Any, Optional
import logging

# Logger spécifique pour l'état
state_logger = logging.getLogger("RhetoricalAnalysisState")
if not state_logger.handlers and not state_logger.propagate:
    handler = logging.StreamHandler(); formatter = logging.Formatter('%(asctime)s [%(levelname)s] [%(name)s] %(message)s', datefmt='%H:%M:%S'); handler.setFormatter(formatter); state_logger.addHandler(handler); state_logger.setLevel(logging.INFO)

class RhetoricalAnalysisState:
    """Représente l'état partagé d'une analyse rhétorique collaborative."""

    # ... (Le code complet de la classe RhetoricalAnalysisState tel que fourni dans la réponse précédente va ici) ...
    # Structure des données de l'état (pour référence)
    raw_text: str
    analysis_tasks: Dict[str, str] # {task_id: description}
    identified_arguments: Dict[str, str] # {arg_id: description}
    identified_fallacies: Dict[str, Dict[str, str]] # {fallacy_id: {type:..., justification:..., target_argument_id?:...}}
    belief_sets: Dict[str, Dict[str, str]] # {bs_id: {logic_type:..., content:...}}
    query_log: List[Dict[str, str]] # [{log_id:..., belief_set_id:..., query:..., raw_result:...}]
    answers: Dict[str, Dict[str, Any]] # {task_id: {author_agent:..., answer_text:..., source_ids:[...]}}
    final_conclusion: Optional[str]
    _next_agent_designated: Optional[str] # Nom de l'agent désigné pour le prochain tour

    def __init__(self, initial_text: str):
        """Initialise un état vide avec le texte brut."""
        self.raw_text = initial_text
        self.analysis_tasks = {}
        self.identified_arguments = {}
        self.identified_fallacies = {}
        self.belief_sets = {}
        self.query_log = []
        self.answers = {}
        self.final_conclusion = None
        self._next_agent_designated = None
        state_logger.debug(f"Nouvelle instance RhetoricalAnalysisState créée (id: {id(self)}) avec texte (longueur: {len(initial_text)}).")

    def _generate_id(self, prefix: str, current_dict_or_list: Any) -> str:
        """Génère un ID simple basé sur la taille actuelle."""
        index = 0
        try:
            if isinstance(current_dict_or_list, (dict, list)):
                index = len(current_dict_or_list)
            else:
                 index = 0
                 state_logger.warning(f"_generate_id: Type inattendu '{type(current_dict_or_list)}' pour prefix '{prefix}'. Index sera 0.")
        except Exception as e:
            state_logger.error(f"Erreur dans _generate_id pour prefix '{prefix}': {e}", exc_info=True)
            index = 999
        safe_index = min(index, 9999)
        return f"{prefix}_{safe_index + 1}"

    def add_task(self, description: str) -> str:
        """Ajoute une tâche d'analyse et retourne son ID."""
        task_id = self._generate_id("task", self.analysis_tasks)
        self.analysis_tasks[task_id] = description
        state_logger.info(f"Tâche ajoutée: {task_id} - '{description[:60]}...'")
        state_logger.debug(f"État tasks après ajout {task_id}: {self.analysis_tasks}")
        return task_id

    def add_argument(self, description: str) -> str:
        """Ajoute un argument identifié et retourne son ID."""
        arg_id = self._generate_id("arg", self.identified_arguments)
        self.identified_arguments[arg_id] = description
        state_logger.info(f"Argument ajouté: {arg_id} - '{description[:60]}...'")
        state_logger.debug(f"État arguments après ajout {arg_id}: {self.identified_arguments}")
        return arg_id

    def add_fallacy(self, fallacy_type: str, justification: str, target_arg_id: Optional[str] = None) -> str:
        """Ajoute un sophisme identifié et retourne son ID."""
        fallacy_id = self._generate_id("fallacy", self.identified_fallacies)
        entry = {"type": fallacy_type, "justification": justification}
        log_target_info = ""
        if target_arg_id:
             if target_arg_id not in self.identified_arguments:
                 state_logger.warning(f"ID argument cible '{target_arg_id}' pour sophisme '{fallacy_id}' non trouvé dans les arguments identifiés ({list(self.identified_arguments.keys())}).")
             entry["target_argument_id"] = target_arg_id
             log_target_info = f" (cible: {target_arg_id})"
        self.identified_fallacies[fallacy_id] = entry
        state_logger.info(f"Sophisme ajouté: {fallacy_id} - Type: {fallacy_type}{log_target_info}")
        state_logger.debug(f"État fallacies après ajout {fallacy_id}: {self.identified_fallacies}")
        return fallacy_id

    def add_belief_set(self, logic_type: str, content: str) -> str:
        """Ajoute un belief set formel et retourne son ID."""
        normalized_type = logic_type.strip().lower().replace(" ", "_")
        bs_id = self._generate_id(f"{normalized_type}_bs", self.belief_sets)
        self.belief_sets[bs_id] = {"logic_type": logic_type, "content": content}
        state_logger.info(f"Belief Set ajouté: {bs_id} - Type: {logic_type}")
        state_logger.debug(f"État belief_sets après ajout {bs_id}: {self.belief_sets}")
        return bs_id

    def log_query(self, belief_set_id: str, query: str, raw_result: str) -> str:
         """Enregistre une requête formelle et son résultat brut."""
         log_id = self._generate_id("qlog", self.query_log)
         if belief_set_id not in self.belief_sets:
             state_logger.warning(f"ID Belief Set '{belief_set_id}' pour query log '{log_id}' non trouvé dans les belief sets ({list(self.belief_sets.keys())}).")
         log_entry = {"log_id": log_id, "belief_set_id": belief_set_id, "query": query, "raw_result": raw_result}
         self.query_log.append(log_entry)
         state_logger.info(f"Requête loggée: {log_id} (sur BS: {belief_set_id}, Query: '{query[:60]}...')")
         state_logger.debug(f"État query_log après ajout {log_id} (taille: {len(self.query_log)}): {self.query_log}")
         return log_id

    def add_answer(self, task_id: str, author_agent: str, answer_text: str, source_ids: List[str]):
        """Ajoute la réponse d'un agent à une tâche spécifiques."""
        if task_id not in self.analysis_tasks:
            state_logger.warning(f"ID Tâche '{task_id}' pour réponse de '{author_agent}' non trouvé dans les tâches définies ({list(self.analysis_tasks.keys())}).")
        self.answers[task_id] = {"author_agent": author_agent, "answer_text": answer_text, "source_ids": source_ids}
        state_logger.info(f"Réponse ajoutée pour tâche '{task_id}' par agent '{author_agent}'.")
        state_logger.debug(f"État answers après ajout réponse pour {task_id}: {self.answers}")

    def set_conclusion(self, conclusion: str):
        """Enregistre la conclusion finale de l'analyse."""
        self.final_conclusion = conclusion
        state_logger.info(f"Conclusion finale enregistrée : '{conclusion[:60]}...'")
        state_logger.debug(f"État final_conclusion après enregistrement: {self.final_conclusion is not None}")

    def designate_next_agent(self, agent_name: str):
        """Désigne l'agent qui doit parler au prochain tour."""
        self._next_agent_designated = agent_name
        state_logger.info(f"Prochain agent désigné: '{agent_name}'")
        state_logger.debug(f"État _next_agent_designated après désignation: '{self._next_agent_designated}'")

    def consume_next_agent_designation(self) -> Optional[str]:
        """Récupère le nom de l'agent désigné et réinitialise la désignation."""
        agent_name = self._next_agent_designated
        self._next_agent_designated = None
        if agent_name:
            state_logger.info(f"Désignation pour '{agent_name}' consommée.")
        return agent_name

    def reset_state(self):
        """Réinitialise l'état à son état initial (vide sauf texte brut)."""
        state_logger.info(">>> Réinitialisation de l'état d'analyse...")
        initial_text = self.raw_text
        self.__init__(initial_text)
        assert not self.analysis_tasks, "Reset analysis_tasks failed"
        assert not self.identified_arguments, "Reset identified_arguments failed"
        assert not self.identified_fallacies, "Reset identified_fallacies failed"
        assert not self.belief_sets, "Reset belief_sets failed"
        assert not self.query_log, "Reset query_log failed"
        assert not self.answers, "Reset answers failed"
        assert self.final_conclusion is None, "Reset final_conclusion failed"
        assert self._next_agent_designated is None, "Reset _next_agent_designated failed"
        state_logger.info("<<< Réinitialisation de l'état terminée et vérifiée.")

    def get_state_snapshot(self, summarize: bool = False) -> Dict[str, Any]:
        """Retourne un dictionnaire représentant l'état actuel (complet ou résumé)."""
        if summarize:
             return {
                 "raw_text_snippet": self.raw_text[:150] + "..." if len(self.raw_text) > 150 else self.raw_text,
                 "task_count": len(self.analysis_tasks),
                 "tasks_defined": list(self.analysis_tasks.keys()),
                 "argument_count": len(self.identified_arguments),
                 "fallacy_count": len(self.identified_fallacies),
                 "belief_set_count": len(self.belief_sets),
                 "query_log_count": len(self.query_log),
                 "answer_count": len(self.answers),
                 "tasks_answered": list(self.answers.keys()),
                 "conclusion_present": self.final_conclusion is not None,
                 "next_agent_designated": self._next_agent_designated
             }
        else:
            return json.loads(self.to_json(indent=None))

    def to_json(self, indent: Optional[int] = 2) -> str:
        """Sérialise l'état actuel en chaîne JSON."""
        state_dict = {k: v for k, v in self.__dict__.items() if not callable(v) and not k.startswith("_logger")}
        try:
            return json.dumps(state_dict, indent=indent, ensure_ascii=False, default=str)
        except TypeError as e:
            state_logger.error(f"Erreur de sérialisation JSON de l'état: {e}")
            safe_dict = {k: repr(v) for k, v in state_dict.items()}
            return json.dumps({"error": f"JSON serialization failed: {e}", "safe_state_repr": safe_dict}, indent=indent)

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'RhetoricalAnalysisState':
        """Crée une instance d'état à partir d'un dictionnaire."""
        state = cls(data.get('raw_text', ''))
        state.analysis_tasks = data.get('analysis_tasks', {})
        state.identified_arguments = data.get('identified_arguments', {})
        state.identified_fallacies = data.get('identified_fallacies', {})
        state.belief_sets = data.get('belief_sets', {})
        state.query_log = data.get('query_log', [])
        state.answers = data.get('answers', {})
        state.final_conclusion = data.get('final_conclusion', None)
        state._next_agent_designated = data.get('_next_agent_designated', None)
        state_logger.debug(f"Instance RhetoricalAnalysisState créée depuis dict (id: {id(state)}).")
        return state

logging.info("Classe RhetoricalAnalysisState définie.")

### 🔌 Classe Plugin : StateManagerPlugin

In [ ]:
# %% CELLULE [3.2] - Définition Classe StateManagerPlugin
# (Remplace une partie de l'ancienne cellule 24085a21)

import json
from typing import Dict, List, Any, Optional
import logging
from semantic_kernel.functions import kernel_function

# Récupérer les loggers et l'état (supposés définis)
# Assurer que le logger StateManager a un handler
sm_logger = logging.getLogger("Orchestration.StateManager")
if not sm_logger.handlers and not sm_logger.propagate:
    handler = logging.StreamHandler(); formatter = logging.Formatter('%(asctime)s [%(levelname)s] [%(name)s] %(message)s', datefmt='%H:%M:%S'); handler.setFormatter(formatter); sm_logger.addHandler(handler); sm_logger.setLevel(logging.INFO)
# S'assurer que la classe état est définie
if 'RhetoricalAnalysisState' not in globals(): raise NameError("Classe RhetoricalAnalysisState non définie.")


class StateManagerPlugin:
    """Plugin Semantic Kernel pour lire et modifier l'état d'analyse partagé."""
    _state: 'RhetoricalAnalysisState' # Référence à l'instance d'état unique
    _logger: logging.Logger

    def __init__(self, state: 'RhetoricalAnalysisState'):
        """Initialise le plugin avec une instance d'état."""
        self._state = state
        self._logger = sm_logger
        self._logger.info(f"StateManagerPlugin initialisé avec l'instance RhetoricalAnalysisState (id: {id(self._state)}).")

    # ... (Le code complet de la classe StateManagerPlugin avec ses @kernel_function va ici) ...
    # ... (Reprendre le code de la réponse précédente pour cette classe) ...
    @kernel_function(description="Récupère un aperçu (complet ou résumé) de l'état actuel de l'analyse.", name="get_current_state_snapshot")
    def get_current_state_snapshot(self, summarize: bool = True) -> str:
        """Retourne l'état actuel sous forme de chaîne JSON."""
        self._logger.info(f"Appel get_current_state_snapshot (state id: {id(self._state)}, summarize={summarize})...")
        try:
            snapshot_dict = self._state.get_state_snapshot(summarize=summarize)
            indent = 2 if not summarize else None
            snapshot_json = json.dumps(snapshot_dict, indent=indent, ensure_ascii=False, default=str)
            self._logger.info(" -> Snapshot de l'état généré avec succès.")
            self._logger.debug(f" -> Snapshot (summarize={summarize}): {snapshot_json[:500] + '...' if len(snapshot_json)>500 else snapshot_json}")
            return snapshot_json
        except Exception as e:
            self._logger.error(f"Erreur lors de la récupération/sérialisation du snapshot de l'état: {e}", exc_info=True)
            return json.dumps({"error": f"Erreur récupération/sérialisation snapshot: {e}"})

    @kernel_function(description="Ajoute une nouvelle tâche d'analyse à l'état.", name="add_analysis_task")
    def add_analysis_task(self, description: str) -> str:
        """Interface Kernel Function pour ajouter une tâche via l'état."""
        self._logger.info(f"Appel add_analysis_task (state id: {id(self._state)}): '{description[:60]}...'")
        try:
            task_id = self._state.add_task(description)
            self._logger.info(f" -> Tâche '{task_id}' ajoutée avec succès via l'état.")
            return task_id
        except Exception as e:
            self._logger.error(f"Erreur lors de l'ajout de la tâche '{description[:60]}...': {e}", exc_info=True)
            return f"FUNC_ERROR: Erreur ajout tâche: {e}"

    @kernel_function(description="Ajoute un argument identifié à l'état.", name="add_identified_argument")
    def add_identified_argument(self, description: str) -> str:
        """Interface Kernel Function pour ajouter un argument via l'état."""
        self._logger.info(f"Appel add_identified_argument (state id: {id(self._state)}): '{description[:60]}...'")
        try:
            arg_id = self._state.add_argument(description)
            self._logger.info(f" -> Argument '{arg_id}' ajouté avec succès via l'état.")
            return arg_id
        except Exception as e:
            self._logger.error(f"Erreur lors de l'ajout de l'argument '{description[:60]}...': {e}", exc_info=True)
            return f"FUNC_ERROR: Erreur ajout argument: {e}"

    @kernel_function(description="Ajoute un sophisme identifié à l'état.", name="add_identified_fallacy")
    def add_identified_fallacy(self, fallacy_type: str, justification: str, target_argument_id: Optional[str] = None) -> str:
        """Interface Kernel Function pour ajouter un sophisme via l'état."""
        self._logger.info(f"Appel add_identified_fallacy (state id: {id(self._state)}): Type='{fallacy_type}', Target='{target_argument_id or 'None'}'...")
        try:
            fallacy_id = self._state.add_fallacy(fallacy_type, justification, target_argument_id)
            self._logger.info(f" -> Sophisme '{fallacy_id}' ajouté avec succès via l'état.")
            return fallacy_id
        except Exception as e:
            self._logger.error(f"Erreur lors de l'ajout du sophisme (Type: {fallacy_type}): {e}", exc_info=True)
            return f"FUNC_ERROR: Erreur ajout sophisme: {e}"

    @kernel_function(description="Ajoute un belief set formel (ex: Propositional) à l'état.", name="add_belief_set")
    def add_belief_set(self, logic_type: str, content: str) -> str:
        """Interface Kernel Function pour ajouter un belief set via l'état. Valide le type logique."""
        self._logger.info(f"Appel add_belief_set (state id: {id(self._state)}): Type='{logic_type}'...")
        valid_logic_types = {"propositional": "Propositional", "pl": "Propositional"}
        normalized_logic_type = logic_type.strip().lower()

        if normalized_logic_type not in valid_logic_types:
            error_msg = f"Type logique '{logic_type}' non supporté. Types valides (insensible casse): {list(valid_logic_types.keys())}"
            self._logger.error(error_msg)
            return f"FUNC_ERROR: {error_msg}"

        validated_logic_type = valid_logic_types[normalized_logic_type]
        try:
            bs_id = self._state.add_belief_set(validated_logic_type, content)
            self._logger.info(f" -> Belief Set '{bs_id}' ajouté avec succès via l'état (Type: {validated_logic_type}).")
            return bs_id
        except Exception as e:
            self._logger.error(f"Erreur interne lors de l'ajout du Belief Set (Type: {validated_logic_type}): {e}", exc_info=True)
            return f"FUNC_ERROR: Erreur interne ajout Belief Set: {e}"

    @kernel_function(description="Enregistre une requête formelle et son résultat brut dans le log de l'état.", name="log_query_result")
    def log_query_result(self, belief_set_id: str, query: str, raw_result: str) -> str:
        """Interface Kernel Function pour logger une requête via l'état."""
        self._logger.info(f"Appel log_query_result (state id: {id(self._state)}): BS_ID='{belief_set_id}', Query='{query[:60]}...'")
        try:
            log_id = self._state.log_query(belief_set_id, query, raw_result)
            self._logger.info(f" -> Requête '{log_id}' loggée avec succès via l'état.")
            return log_id
        except Exception as e:
            self._logger.error(f"Erreur lors du logging de la requête (BS_ID: {belief_set_id}): {e}", exc_info=True)
            return f"FUNC_ERROR: Erreur logging requête: {e}"

    @kernel_function(description="Ajoute une réponse d'un agent à une tâche d'analyse spécifique dans l'état.", name="add_answer")
    def add_answer(self, task_id: str, author_agent: str, answer_text: str, source_ids: List[str]) -> str:
        """Interface Kernel Function pour ajouter une réponse via l'état."""
        self._logger.info(f"Appel add_answer (state id: {id(self._state)}): TaskID='{task_id}', Author='{author_agent}'...")
        try:
            self._state.add_answer(task_id, author_agent, answer_text, source_ids)
            self._logger.info(f" -> Réponse pour tâche '{task_id}' ajoutée avec succès via l'état.")
            return f"OK: Réponse pour {task_id} ajoutée."
        except Exception as e:
            self._logger.error(f"Erreur lors de l'ajout de la réponse pour la tâche '{task_id}': {e}", exc_info=True)
            return f"FUNC_ERROR: Erreur ajout réponse pour {task_id}: {e}"

    @kernel_function(description="Enregistre la conclusion finale de l'analyse dans l'état.", name="set_final_conclusion")
    def set_final_conclusion(self, conclusion: str) -> str:
        """Interface Kernel Function pour enregistrer la conclusion via l'état."""
        self._logger.info(f"Appel set_final_conclusion (state id: {id(self._state)}): '{conclusion[:60]}...'")
        try:
            self._state.set_conclusion(conclusion)
            self._logger.info(f" -> Conclusion finale enregistrée avec succès via l'état.")
            return "OK: Conclusion finale enregistrée."
        except Exception as e:
            self._logger.error(f"Erreur lors de l'enregistrement de la conclusion finale: {e}", exc_info=True)
            return f"FUNC_ERROR: Erreur enregistrement conclusion: {e}"

    @kernel_function(description="Désigne quel agent doit parler au prochain tour. Utiliser le nom EXACT de l'agent.", name="designate_next_agent")
    def designate_next_agent(self, agent_name: str) -> str:
        """Interface Kernel Function pour désigner le prochain agent via l'état."""
        self._logger.info(f"Appel designate_next_agent (state id: {id(self._state)}): Prochain = '{agent_name}'")
        try:
            self._state.designate_next_agent(agent_name)
            self._logger.info(f" -> Agent '{agent_name}' désigné avec succès via l'état.")
            return f"OK. Agent '{agent_name}' désigné pour le prochain tour."
        except Exception as e:
            self._logger.error(f"Erreur lors de la désignation de l'agent '{agent_name}': {e}", exc_info=True)
            return f"FUNC_ERROR: Erreur désignation agent {agent_name}: {e}"


logging.info("Classe StateManagerPlugin définie.")


### ⚙️ Création : Service LLM Global

In [ ]:
# %% CELLULE [3.3] - Création Service LLM Global
# (Remplace une partie de l'ancienne cellule 24085a21)

import logging
import os
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, AzureChatCompletion

# Récupérer loggers et variables de config (supposés définis)
llm_logger = logging.getLogger("Orchestration.LLM")
if not llm_logger.handlers and not llm_logger.propagate: # Assurer handler
     handler = logging.StreamHandler(); formatter = logging.Formatter('%(asctime)s [%(levelname)s] [%(name)s] %(message)s', datefmt='%H:%M:%S'); handler.setFormatter(formatter); llm_logger.addHandler(handler); llm_logger.setLevel(logging.INFO)

if 'api_key' not in globals() or 'model_id' not in globals() or 'use_azure_openai' not in globals():
     raise RuntimeError("Variables de configuration LLM non trouvées. Exécutez la cellule [1].")

global_ai_service_instance = None
llm_logger.info("--- Configuration du Service LLM Global ---")

try:
    if use_azure_openai:
        llm_logger.info("Configuration Service Global: AzureChatCompletion...")
        if 'endpoint' not in globals() or not endpoint: raise ValueError("Endpoint Azure manquant.")
        global_ai_service_instance = AzureChatCompletion(
            service_id="global_llm_service",
            deployment_name=model_id,
            endpoint=endpoint,
            api_key=api_key
        )
        llm_logger.info(f"Service LLM global Azure ({model_id}) créé.")
    else:
        llm_logger.info("Configuration Service Global: OpenAIChatCompletion...")
        if 'org_id' not in globals(): org_id = None # Assurer existence
        global_ai_service_instance = OpenAIChatCompletion(
            service_id="global_llm_service",
            ai_model_id=model_id,
            api_key=api_key,
            org_id=org_id
        )
        llm_logger.info(f"Service LLM global OpenAI ({model_id}) créé.")
except Exception as e:
    llm_logger.critical(f"Erreur critique lors de la création du service LLM global: {e}", exc_info=True)
    raise RuntimeError(f"Impossible de configurer le service LLM global: {e}")

if not global_ai_service_instance:
     raise RuntimeError("Configuration du service LLM global a échoué silencieusement.")

logging.info("--- Fin Définitions Composants Partagés ---")

# Rappel: PAS d'instance de RhetoricalAnalysisState ou StateManagerPlugin ou Kernel ici.

## 4. Agent : 🧑‍🏫 ProjectManagerAgent (Définitions)

Cet agent est responsable de l'orchestration globale de l'analyse.

**Rôle :**
*   Analyser la demande initiale et l'état actuel (`StateManager.get_current_state_snapshot`).
*   Définir les tâches d'analyse pour les agents spécialistes (`StateManager.add_analysis_task` via `PM.semantic_DefineTasksAndDelegate`).
*   Assigner les tâches et désigner le prochain agent à intervenir (`StateManager.designate_next_agent`). **Attention:** Doit utiliser le nom exact de l'agent (e.g., `PropositionalLogicAgent`).
*   Suivre l'avancement en consultant l'état (tâches définies vs tâches répondues).
*   Synthétiser les résultats une fois les analyses pertinentes terminées (`StateManager.set_final_conclusion` via `PM.semantic_WriteAndSetConclusion`).

**Composants Définis Ci-dessous :**
*   `ProjectManagerPlugin` (Classe)
*   Prompts Sémantiques (`prompt_define_tasks_v10`, `prompt_write_conclusion_v6`)
*   Fonction de configuration `setup_pm_kernel`
*   Instructions Système `PM_INSTRUCTIONS` (V8)

### 🔌 Classe Plugin et 📜 Prompts Sémantiques (PM)

In [ ]:
# %% CELLULE [4.1] - PM Plugin et Prompts Sémantiques
# (Remplace une partie de l'ancienne cellule a5621670)

import logging

logger = logging.getLogger("Orchestration.AgentPM.Defs")

# --- Plugin Spécifique PM (Vide actuellement) ---
class ProjectManagerPlugin:
    """Plugin pour fonctions natives spécifiques au Project Manager (si nécessaire)."""
    pass

# --- Fonctions Sémantiques PM ---

# Aide à la planification (V10 - Correction nom agent désigné)
prompt_define_tasks_v10 = """
[Contexte]
Vous êtes le ProjectManagerAgent. Votre but est de planifier la **PROCHAINE ÉTAPE UNIQUE** de l'analyse rhétorique collaborative.
Agents disponibles et leurs noms EXACTS:
- "ProjectManagerAgent" (Vous-même, pour conclure)
- "InformalAnalysisAgent" (Identifie arguments OU analyse sophismes via taxonomie CSV)
- "PropositionalLogicAgent" (Traduit texte en PL OU exécute requêtes logiques PL via Tweety)

[État Actuel (Snapshot JSON)]
{{$analysis_state_snapshot}}

[Texte Initial (pour référence)]
{{$raw_text}}

[Séquence d'Analyse Idéale (si applicable)]
1. Identification Arguments ("InformalAnalysisAgent")
2. Analyse Sophismes ("InformalAnalysisAgent" - **LIMITÉE à 2-3 tours maximum**)
3. Traduction en Belief Set PL ("PropositionalLogicAgent") 
4. Exécution Requêtes PL ("PropositionalLogicAgent")
5. Conclusion (Vous-même, "ProjectManagerAgent")

[Instructions CRITIQUES]
1.  **Analysez l'état CRITIQUEMENT :** Quelles tâches (`tasks_defined`) existent ? Lesquelles ont une réponse (`tasks_answered`) ? Y a-t-il une `final_conclusion` ?
2.  **LOGIQUE DE PROGRESSION FORCÉE** - Déterminez la prochaine étape selon CES RÈGLES STRICTES :
    
    **ÉTAPE 1 - Arguments :**
    - Si AUCUNE tâche contenant "argument" dans tasks_answered → "Identifier les arguments". Agent: "InformalAnalysisAgent"
    
    **ÉTAPE 2 - Sophismes (LIMITÉE) :**
    - Si arguments identifiés ET (aucune tâche "sophisme" OU moins de 3 tâches "sophisme" répondues) ET pas de belief_sets → "Analyser les sophismes (commencer par exploration racine PK=0)". Agent: "InformalAnalysisAgent"
    
    **ÉTAPE 3 - Translation PL (FORCÉE après sophismes) :**
    - Si arguments identifiés ET (au moins 1 tâche "sophisme" répondue OU 3+ tâches "sophisme" total) ET belief_sets vide → "Traduire les arguments en logique propositionnelle". Agent: "PropositionalLogicAgent"
    
    **ÉTAPE 4 - Requêtes PL :**
    - Si belief_sets non-vide ET aucune requête dans query_log → "Exécuter des requêtes logiques sur le belief set". Agent: "PropositionalLogicAgent"
    
    **ÉTAPE 5 - Conclusion :**
    - Si toutes étapes précédentes faites ET final_conclusion null → Conclusion

3.  **RÈGLES STRICTES DE NON-BLOCAGE :**
    - **JAMAIS plus de 3 tâches sophismes** - Forcer passage à PL après
    - **NE PAS répéter** une même étape si elle a déjà 2+ réponses
    - **PROGRESSION OBLIGATOIRE** - Si sophismes ont commencé, finir par PL
    
4.  **Formulez UN SEUL appel** `StateManager.add_analysis_task` avec la description exacte.
5.  **Formulez UN SEUL appel** `StateManager.designate_next_agent` avec le **nom EXACT**.
6.  Message délégation: "[NomAgent EXACT], veuillez effectuer la tâche [ID]: [Description]."

[Sortie Attendue]
Plan: [Prochaine étape avec justification progression]
Appels:
1. StateManager.add_analysis_task(description="[Description]") # ID task_N
2. StateManager.designate_next_agent(agent_name="[Nom Exact Agent]")
Message de délégation: "[Agent], veuillez effectuer la tâche task_N: [Description]"
"""

# Aide à la conclusion (V6)
prompt_write_conclusion_v6 = """
[Contexte]
Vous êtes le ProjectManagerAgent. On vous demande de conclure l'analyse.
Votre but est de synthétiser les résultats et enregistrer la conclusion.

[État Final de l'Analyse (Snapshot JSON)]
{{$analysis_state_snapshot}}

[Texte Initial (pour référence)]
{{$raw_text}}

[Instructions]
1.  **Vérification PRÉALABLE OBLIGATOIRE :** Examinez l'état (`analysis_state_snapshot`). L'analyse semble-t-elle _raisonnablement complète_ ?
    *   Y a-t-il des `identified_arguments` ? (Indispensable)
    *   Y a-t-il des réponses (`answers`) pour les tâches clés comme l'identification d'arguments, l'analyse de sophismes (si effectuée), la traduction PL (si effectuée) ?
    *   **Si l'analyse semble manifestement incomplète (ex: pas d'arguments identifiés, ou une tâche majeure sans réponse), NE PAS CONCLURE.** Répondez: "ERREUR: Impossible de conclure, l'analyse semble incomplète. Vérifiez l'état." et n'appelez PAS `StateManager.set_final_conclusion`.
2.  **Si la vérification est OK :** Examinez TOUS les éléments pertinents de l'état final : `identified_arguments`, `identified_fallacies` (si présents), `belief_sets` et `query_log` (si présents), `answers` (pour le contenu des analyses).
3.  Rédigez une conclusion synthétique et nuancée sur la rhétorique du texte, basée EXCLUSIVEMENT sur les informations de l'état.
4.  Formulez l'appel à `StateManager.set_final_conclusion` avec votre texte de conclusion.

[Sortie Attendue (si conclusion possible)]
Fournissez la conclusion rédigée, puis l'appel de fonction formaté.
Conclusion:
[Votre synthèse ici]
Appel:
StateManager.set_final_conclusion(conclusion="[Copie de votre synthèse ici]")

[Sortie Attendue (si conclusion impossible)]
ERREUR: Impossible de conclure, l'analyse semble incomplète. Vérifiez l'état.
"""

logger.info("Plugin PM (vide) et prompts sémantiques (V10, V6) définis.")


### ⚙️ Fonction : setup_pm_kernel

In [ ]:
# %% CELLULE [4.2] - Fonction setup_pm_kernel
# (Remplace une partie de l'ancienne cellule a5621670)

import semantic_kernel as sk
import logging

# S'assurer que les dépendances sont là
if 'ProjectManagerPlugin' not in globals(): raise NameError("Classe ProjectManagerPlugin non définie.")
if 'prompt_define_tasks_v10' not in globals(): raise NameError("Prompt prompt_define_tasks_v10 non défini.")
if 'prompt_write_conclusion_v6' not in globals(): raise NameError("Prompt prompt_write_conclusion_v6 non défini.")

logger = logging.getLogger("Orchestration.AgentPM.Setup")

def setup_pm_kernel(kernel: sk.Kernel, llm_service):
    """Ajoute le plugin PM et ses fonctions sémantiques au kernel donné."""
    plugin_name = "PM"
    logger.info(f"Configuration Kernel pour {plugin_name} (V10 - Fix Désignation)...")

    if plugin_name not in kernel.plugins:
        kernel.add_plugin(ProjectManagerPlugin(), plugin_name=plugin_name)
        logger.debug(f"Plugin natif '{plugin_name}' ajouté au kernel PM.")
    else:
        logger.debug(f"Plugin natif '{plugin_name}' déjà présent dans le kernel PM.")

    default_settings = None
    if llm_service:
        try:
            default_settings = kernel.get_prompt_execution_settings_from_service_id(llm_service.service_id)
            logger.debug(f"Settings LLM récupérés pour {plugin_name}.")
        except Exception as e:
            logger.warning(f"Impossible de récupérer les settings LLM pour {plugin_name}: {e}")

    try:
        kernel.add_function(
            prompt=prompt_define_tasks_v10,
            plugin_name=plugin_name, function_name="semantic_DefineTasksAndDelegate",
            description="Définit la PROCHAINE tâche unique, l'enregistre, désigne 1 agent (Nom Exact Requis).",
            prompt_execution_settings=default_settings
        )
        logger.debug(f"Fonction {plugin_name}.semantic_DefineTasksAndDelegate (V10) ajoutée/mise à jour.")
    except ValueError as ve: logger.warning(f"Problème ajout/MàJ {plugin_name}.semantic_DefineTasksAndDelegate: {ve}")

    try:
        kernel.add_function(
            prompt=prompt_write_conclusion_v6,
            plugin_name=plugin_name, function_name="semantic_WriteAndSetConclusion",
            description="Rédige/enregistre conclusion finale (avec pré-vérification état).",
            prompt_execution_settings=default_settings
        )
        logger.debug(f"Fonction {plugin_name}.semantic_WriteAndSetConclusion (V6) ajoutée/mise à jour.")
    except ValueError as ve: logger.warning(f"Problème ajout/MàJ {plugin_name}.semantic_WriteAndSetConclusion: {ve}")

    logger.info(f"Kernel {plugin_name} configuré (V10).")


### 📜 Instructions Système : PM_INSTRUCTIONS

In [ ]:
# %% CELLULE [4.3] - Instructions Système PM
# (Remplace une partie de l'ancienne cellule a5621670)

import logging

logger = logging.getLogger("Orchestration.AgentPM.Instructions")

# Instructions Système PM (V8 - Correction Noms Agents)
PM_INSTRUCTIONS_V8 = """
Votre Rôle: Chef d'orchestre. Vous devez coordonner les autres agents.
**Noms Exacts des Agents à utiliser pour la désignation:** "InformalAnalysisAgent", "PropositionalLogicAgent".

**Processus OBLIGATOIRE:**

1.  **CONSULTER ÉTAT:** Appelez `StateManager.get_current_state_snapshot(summarize=True)`. Analysez **minutieusement** `tasks_defined`, `tasks_answered`, `final_conclusion`, et les derniers éléments ajoutés.
2.  **DÉCIDER ACTION:**
    *   **A. Tâche Suivante?** Si une étape logique de la séquence (Args -> Sophismes -> PL Trad -> PL Query) est terminée (tâche correspondante dans `tasks_answered`) ET que la suivante n'a pas été lancée OU si aucune tâche n'existe :
        1.  Appelez `StateManager.get_current_state_snapshot(summarize=False)` (`snapshot_json`).
        2.  Appelez `PM.semantic_DefineTasksAndDelegate` en passant `analysis_state_snapshot=snapshot_json` et `raw_text=[Contenu texte]`. **Suivez STRICTEMENT le format de sortie et utilisez les NOMS EXACTS des agents ("InformalAnalysisAgent", "PropositionalLogicAgent").** Ne générez qu'UNE tâche et UNE désignation.
        3.  Formulez le message texte de délégation EXACTEMENT comme indiqué par `PM.semantic_DefineTasksAndDelegate`.
    *   **B. Attente?** Si une tâche définie (`tasks_defined`) N'EST PAS dans `tasks_answered` -> Réponse: "J'attends la réponse de [Agent Probable] pour la tâche [ID Tâche manquante]." **NE PAS DEFINIR de nouvelle tâche.**
    *   **C. Fin?** Si TOUTES les étapes d'analyse pertinentes (Arguments, Sophismes, PL si pertinent) semblent terminées (vérifiez les `answers` pour les tâches correspondantes) ET `final_conclusion` est `null`:
        1. Appelez `StateManager.get_current_state_snapshot(summarize=False)` (`snapshot_json`).
        2. Appelez `PM.semantic_WriteAndSetConclusion(analysis_state_snapshot=snapshot_json, raw_text=[Contenu texte])`.
        3. Formulez un message indiquant que la conclusion est prête et enregistrée.
    *   **D. Déjà Fini?** Si `final_conclusion` n'est PAS `null` -> Réponse: "L'analyse est déjà terminée."

**Règles CRITIQUES:**
*   Pas d'analyse personnelle. Suivi strict de l'état (tâches/réponses).
*   **Utilisez les noms d'agent EXACTS** ("InformalAnalysisAgent", "PropositionalLogicAgent") lors de la désignation via `StateManager.designate_next_agent`.
*   Format de délégation strict.
*   **UNE SEULE** tâche et **UNE SEULE** désignation par étape de planification.
*   Ne concluez que si TOUT le travail pertinent est fait et vérifié dans l'état.
"""

# Utiliser les nouvelles instructions lors de l'instanciation de l'agent
PM_INSTRUCTIONS = PM_INSTRUCTIONS_V8

logger.info("Instructions Système PM_INSTRUCTIONS (V8) définies.")

# --- PAS D'INSTANCIATION D'AGENT ICI ---